<a href="https://colab.research.google.com/github/Namtk214/Images-Retrival/blob/main/Images_Retrival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
ROOT = '/content/drive/MyDrive/Ảnh thẻ Ngô Thành Nam/data/data'
CLASS_NAME = sorted(list(os.listdir(f'{ROOT}/train')))
print(CLASS_NAME)

['African_crocodile', 'American_egret', 'Doberman', 'Lakeland_terrier', 'Orange_easy', 'Rhodesian_ridgeback', 'ambulance', 'apiary', 'bannister', 'barn_spider', 'basketball', 'black_widow', 'brain_coral', 'bullet_train', 'can_opener', 'capuchin', 'car_mirror', 'carousel', 'castle', 'china_cabinet', 'cliff_dwelling', 'comic_book', 'conch', 'cornet', 'dugong', 'electric_locomotive', 'flamingo', 'flatworm', 'goldfish', 'grocery_store', 'guillotine', 'half_track', 'hen-of-the-woods', 'horizontal_bar', 'killer_whale', 'kit_fox', 'knee_pad', 'lion', 'loudspeaker', 'lynx', 'magpie', 'meat_loaf', 'mixing_bowl', 'parachute', 'pizza', 'ram', 'red-backed_sandpiper', 'ruffed_grouse', 'safety_pin', 'scabbard', 'screen', 'steam_locomotive', 'steel_arch_bridge', 'theater_curtain', 'traffic_light', 'triceratops', 'vine_snake', 'warplane', 'white_stork', 'yawl']


In [ ]:
def read_image_from_path(path, size):
  im = Image.open(path).convert('RGB').resize(size)
  return np.array(im)

In [ ]:
def folder_to_image(folder, size):
  list_dir = [folder + '/' + i for i in os.listdir(folder)]
  images_np = np.zeros(shape=(len(list_dir), *size, 3))
  images_path = []
  for i, path in enumerate(list_dir):
    images_np[i] = read_image_from_path(path, size)
    images_path.append(path)
  images_path = np.array(images_path)
  return images_np, images_path

In [ ]:
def absolute_difference(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.sum(np.abs(query - data), axis=axis_batch_size)
#

In [ ]:
def get_l1_score(root_imp_path, query_path, size):
  query  = read_image_from_path(query_path, size)
  ls_path_score = []
  for folder in os.listdir(root_imp_path):
    if folder in CLASS_NAME:
      path = root_imp_path + folder
      images_np, images_path = folder_to_image(path, size)
      rates = absolute_difference(query, images_np)
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score
#

In [ ]:
def mean_square_difference(query_data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.sum(np.square(query - data), axis=axis_batch_size)

In [ ]:
#trả về pattern tương đồng
def consine_similarity(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.sum(query * data, axis=axis_batch_size)

In [ ]:
def correlation_coefficient(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.sum(query * data, axis=axis_batch_size) / np.sqrt(np.sum(query ** 2) * np.sum(data ** 2))

In [ ]:
!pip install chromadb
!pip install open-clip-torch
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.6 MB/s eta 0:00:00
 

In [ ]:
embedding_function = OpenCLIPEmbeddingFunction()
def get_single_image_embedding(image):
  embedding = embedding_function._encode_image(image=image)
  return np.array(embedding)